In [1]:
import socket
import time
import select

class AMDC:
    def __init__(self, 
                 amdc_ip_addr = '192.168.1.10',
                 amdc_ip_port = 7):
        # Serial port configuration
        self.amdc_ip_addr = amdc_ip_addr
        self.amdc_ip_port = amdc_ip_port
        
        self.sockets = []

    def new_socket(self, socket_type):
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.connect((self.amdc_ip_addr, self.amdc_ip_port))
        
        s.setblocking(0)
        
        if socket_type in ['ascii_cmd']:
            s.send(bytearray([12, 34]))
        elif socket_type in ['log_var']:
            s.send(bytearray([56, 78]))
        else:
            raise NameError('InvalidSocketType')
        
        self.sockets.append(s)
        return s
    
    def close_all_sockets(self):
        for s in self.sockets:
            s.close()
    
    @staticmethod
    def cmd(socket, cmd_str):
        socket.send(str.encode(cmd_str))
    
    @staticmethod
    def cmdresp(socket, cmd_str, timeout_sec=0.3):
        socket.send(str.encode(cmd_str))
        
        retdata = []
        
        loops = 5
        for i in range(0,loops):
            ready = select.select([socket], [], [], timeout_sec/loops)
            if ready[0]:
                retdata.append(socket.recv(4096))
        
        return retdata

In [2]:
amdc = AMDC()

In [3]:
s1 = amdc.new_socket('ascii_cmd')

In [144]:
AMDC.cmd(s1, 'blink init\n')

In [11]:
data = AMDC.cmdresp(s1, 'blink init\n')
data

[b'blink init\r\nSUCCESS\r\n\n']

In [482]:
# Ha, we can run this and the AMDC does not miss a BEAT.
# 
# The host core is buffering all the incoming TCP packet
# data and then the ICC code is slowly pushing the data
# to the other core, 1 byte every 100 usec...
#
# This is awesome :)
for i in range(0,1000):
    AMDC.cmd(s1, 'blink deinit\n')
    AMDC.cmd(s1, 'hw led set 0 255 0 0\n')
    AMDC.cmd(s1, 'hw led set 1 0 255 0\n')
    AMDC.cmd(s1, 'hw led set 2 0 0 255\n')
    AMDC.cmd(s1, 'hw led set 3 255 255 255\n')
    AMDC.cmd(s1, 'blink init\n')

In [566]:
amdc.close_all_sockets()

In [10]:
import select

s1.setblocking(0)

timeout_in_seconds = 0.200
ready = select.select([s1], [], [], timeout_in_seconds)
data = []
if ready[0]:
    data = s1.recv(4096)
    
data

[]